Relier la borne de tension de sortie Vout du capteur de pH à la borne de mesure EA0 de la carte Arduino.

Le programme à envoyer à Arduino est acquisition_continue.ino

On commence par importer les bibliothèques nécessaires. Attention : NE PAS EFFACER !

In [ ]:
%run Import_bibliotheques.ipynb
%run Arduino.ipynb
connexion_arduino()

In [ ]:
input("Mettre la sonde dans la solution tampon de pH 7 et appuyer sur entrée")
# Réinitialisation de l'Arduino et attente de la première ligne valide
reset_arduino()
tension7 = mesure_unique()
input("Mettre la sonde dans la solution tampon de pH 4 et appuyer sur entrée")
# Réinitialisation de l'Arduino et attente de la première ligne valide
reset_arduino()
tension4 = mesure_unique()
a = 3. * (tension7 - tension4)
b = 7. - a * tension7

In [ ]:
valeurs = "V,pH"
while 1 :
    try :
        V = float(input("Entrer le volume versé en mL (entrer une ligne vide pour arrêter) : "))
        valeurs += f"\n{V},"
        # Réinitialisation de l'Arduino et attente de la première ligne valide
        reset_arduino()
        tension = mesure_unique()
        
        pHactuel = a * tension + b
        print(f"pH mesuré : {pHactuel:.2f}")
        valeurs += f"{pHactuel}"
    except :
        ser.close()
        
        fichier = open("titragepH.csv", 'w')
        fichier.write(valeurs)
        fichier.close()
        break

Importation de tableaux enregistrés localement en fichiers csv :

In [ ]:
tableurVersVariables('titragepH.csv')

On lisse les valeurs de pH et on dérive numériquement le pH par rapport au volume versé :

In [ ]:
tck = interpolate.splrep(V, pH, s=0.003) # la fonction renvoie un tuple contenant les informations du lissage
# le paramètre 's' permet d'ajuster le lissage : s=0 la courbe passe obligatoirement par tous les points.
# on peut ajuster le lissage de sorte que la dérivée ne soit pas trop bruitée

vecV = linspace(0,max(V),1000) # On définit un vecteur de volumes régulièrement espacés pour le tracé et le calcul de la dérivée
pHLisse = interpolate.splev(vecV,tck) # On calcule le lissage des points de mesures

dervpH = gradient(pHLisse) / gradient(vecV) # On calcule la dérivée du pH par rapport au volume
dervpH = dervpH/max(dervpH)*10 # on normalise la valeur de la dérivée pour l'ajuste à l'affichage

Tracer de graphique(s) à partir des tableaux précédents :

In [ ]:
subplot(1, 1, 1)    # lignes, colonnes, numéro du graphique

plot(V, pH, '+', label="pH mesurés")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe
plot(vecV, pHLisse, '-', label="courbe lissée de pH")
plot(vecV, dervpH, '-', label="dpH/dV") 

grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Courbe de titrage")    # mettre le titre désiré entre les guillemets
xlabel("Volume versé (mL)")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("pH ; dpH/dV");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.